In [1]:

from collections import Counter
import numpy as np
import pandas as pd 
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from sklearn.manifold import MDS
exec(compile(open(r"../nsimplices.py", encoding="utf8").read(), "nsimplices.py", 'exec'))

In [2]:
df_gene = pd.read_csv("~/nSimplices/data/sce_full_Darmanis_scScope.csv", index_col=0, header=0)
df_gene.shape

df_meta = pd.read_csv("~/nSimplices/data/mouse_metadata.csv", index_col=0, header=0)

In [3]:

### Run nSimplices method
feature_num = df_gene.shape[1]
dim_start = 2
# dim_end = df_gene.shape[1]
dim_end = 2
out_dis = pdist(df_gene)
out_dis_sq = squareform(out_dis)

outlier_indices, subspace_dim, corr_dis_sq, corr_coord = nsimplices(out_dis_sq, feature_num, dim_start, dim_end, euc_coord=np.array(df_gene.copy()))

dim in find_subspace_dim is: 2
med_height is: [20414.09755605]
subspace_dim is: 2
thres is: 38770.5663004621 mean is: 23835.329678941573 std is: 4978.412207173509
outlier indices are: [ 90  91  94 102 103 108 114 145 148 160 200 205 206 208 209 214 217 220
 226 231 239 245 249 250 251 252 284 295 297 321 323 325 329 330 332 339
 353 355 397 401 404 407 411 413]
idx is: 90 height is: 40141.23525118166 thres is: 38770.5663004621
idx is: 91 height is: 43400.38562647667 thres is: 38770.5663004621
idx is: 94 height is: 55516.5950350355 thres is: 38770.5663004621
idx is: 102 height is: 42699.393873860354 thres is: 38770.5663004621
idx is: 103 height is: 42841.24651963591 thres is: 38770.5663004621
idx is: 108 height is: 40282.45081275693 thres is: 38770.5663004621
idx is: 114 height is: 46899.308119386056 thres is: 38770.5663004621
idx is: 145 height is: 41652.36594490554 thres is: 38770.5663004621
idx is: 148 height is: 45751.091897910985 thres is: 38770.5663004621
idx is: 160 height is: 43

In [4]:
print(outlier_indices)

[ 90  91  94 102 103 108 114 145 148 160 200 205 206 208 209 214 217 220
 226 231 239 245 249 250 251 252 284 295 297 321 323 325 329 330 332 339
 353 355 397 401 404 407 411 413]


In [5]:
# Percent of outliers 
print(df_gene.shape)
print(len(outlier_indices))

(420, 17206)
44


In [6]:
def reduce_dim(coords):
    # assume that optimal dimension is 2 from DeCOr-MDS

    pca = PCA(n_components=2) 
    new_coords = pca.fit_transform(coords)

    # embedding = MDS(n_components=9)
    # new_coords = embedding.fit_transform(coords)

    return new_coords

In [7]:
def cluster(coords, true_labels, random_state):
    """ 
    Apply K-means, decide cluster IDs
    """
    n_clusters = len(list(set(true_labels)))
    # random state = 3, 6, 7 works better 
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state).fit(coords)
    return kmeans.labels_


In [8]:
def infer_cluster_labels(cluster_ids, true_labels):
    """ 
    Infer one label for each cluster 
    """
    def majority_label(labels):
        ''' 
        Infer labels by majority voting
        '''
        c = Counter(labels)
        value, _ = c.most_common()[0]
        return value
    pred_labels = np.empty(len(true_labels), dtype=object)
    unique_ids = list(set(cluster_ids))
    for id in unique_ids:
        id_indices = [i for i, x in enumerate(cluster_ids) if x == id]
        id_true_labels = true_labels[id_indices]
        pred_label = majority_label(id_true_labels)
        pred_labels[id_indices] = pred_label
    return pred_labels


In [9]:
def cluster_proc(coords, true_labels, random_state):
    """ 
    @param coords np.array: raw coordinates or corrected coordinates from DeCOr-MDS
    """
    cluster_ids = cluster(coords, true_labels, random_state)
    pred_labels = infer_cluster_labels(cluster_ids, true_labels)
    NMI = normalized_mutual_info_score(true_labels, pred_labels)
    ARI = adjusted_rand_score(true_labels, pred_labels)
    return NMI, ARI

In [10]:
true_labels = np.array(df_meta["cell_type"])
coords = reduce_dim(np.array(df_gene))
avg_NMI = 0
avg_ARI = 0
avg_base_NMI = 0
avg_base_ARI = 0
for i in range(10):
    print("random state:", i)
    NMI, ARI = cluster_proc(corr_coord, true_labels, i)
    base_NMI, base_ARI = cluster_proc(coords, true_labels, i)
    print(f"NMI: {NMI}, base_NMI, {base_NMI}, ARI, {ARI}, base_ARI: {base_ARI}")
    avg_NMI += NMI 
    avg_ARI += ARI
    avg_base_NMI += base_NMI
    avg_base_ARI += base_ARI
print(f"avg_NMI: {avg_NMI/10}, avg_base_NMI: {avg_base_NMI/10}, avg_ARI: {avg_ARI/10}, avg_base_ARI: {avg_base_ARI/10}")


random state: 0


TypeError: '<' not supported between instances of 'NoneType' and 'str'